# Cleaning Up

This notebook demonstrates how to clean up all the resources created in the previous set of notebooks.

In [1]:
import boto3
import json

In [2]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

In [3]:
%store -r

Please note these steps rely on each step being fully completed before the other will work. If you receive an error, wait a minute or two and try again.

## Clean up Campaigns

Start by cleaning up the campaigns. You need to delete campaigns before you can delete the solution versions upon which they are based. 

The code below will delete the Campaigns you created.

In [4]:
personalize.delete_campaign(
    campaignArn = userpersonalization_campaign_arn
)
personalize.delete_campaign(
    campaignArn = sims_campaign_arn
)
personalize.delete_campaign(
    campaignArn = rerank_campaign_arn
)

NameError: name 'sims_campaign_arn' is not defined

## Clean up event trackers

Next, clean up any event trackers. The code below will delete your event tracker.

In [5]:
personalize.delete_event_tracker(
    eventTrackerArn = event_tracker_arn
)

{'ResponseMetadata': {'RequestId': '29c451b9-3412-4262-a247-8466a499371d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 01 Feb 2021 19:08:35 GMT',
   'x-amzn-requestid': '29c451b9-3412-4262-a247-8466a499371d',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Clean up Filters

Next, clean up any filters. The code below iterate over all filters you created and delete them. Note, if you get an error about a campaign, that is fine, wait for 2 minutes then try the cell below again. 

Note: You will not be able to delete the filters until the campaign deletion completes, which may take a few minutes

In [11]:
personalize.delete_filter(filterArn = genre_filter_arn)

for filter_arn in interaction_filter_arns:
    personalize.delete_filter(
        filterArn = filter_arn
    )
for filter_arn in meta_filter_decade_arns:
    personalize.delete_filter(
        filterArn = filter_arn
    )

## Clean up solutions

Next, clean up the solutions. The code below will delete the solutions that were created.

In [12]:
personalize.delete_solution(
    solutionArn = user_personalization_solution_arn
)
personalize.delete_solution(
    solutionArn = sims_solution_arn
)
personalize.delete_solution(
    solutionArn = rerank_solution_arn
)

{'ResponseMetadata': {'RequestId': '263b4618-cf59-4fe9-8a47-ed98c37ae79c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 01 Feb 2021 19:15:01 GMT',
   'x-amzn-requestid': '263b4618-cf59-4fe9-8a47-ed98c37ae79c',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Clean up datasets

Next, clean up the datasets. The code below will delete the item and interactions datasets.

In [14]:
personalize.delete_dataset(
    datasetArn = items_dataset_arn
)
personalize.delete_dataset(
    datasetArn = interactions_dataset_arn
)

{'ResponseMetadata': {'RequestId': '721fef18-ed12-437e-af7e-2df7978f321c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 01 Feb 2021 19:15:34 GMT',
   'x-amzn-requestid': '721fef18-ed12-437e-af7e-2df7978f321c',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Clean up the schemas

Next, clean up the schemas. You create an interaction and item schema, delete them.

In [16]:
personalize.delete_schema(
    schemaArn = interaction_schema_arn
)
personalize.delete_schema(
    schemaArn = itemmetadataschema_arn
)

{'ResponseMetadata': {'RequestId': '08740ae8-e723-49bd-b09b-581d92bda9a4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 01 Feb 2021 19:16:17 GMT',
   'x-amzn-requestid': '08740ae8-e723-49bd-b09b-581d92bda9a4',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Clean up the dataset groups

Finally, clean up the dataset group:

In [17]:
personalize.delete_dataset_group(
    datasetGroupArn = dataset_group_arn
)

{'ResponseMetadata': {'RequestId': 'd6e8af56-e334-40f4-882b-ead3f300bccf',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 01 Feb 2021 19:16:20 GMT',
   'x-amzn-requestid': 'd6e8af56-e334-40f4-882b-ead3f300bccf',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Clean up the S3 bucket and IAM role

Start by deleting the role, then empty the bucket, then delete the bucket.

In [18]:
iam = boto3.client('iam')

Identify the name of the role you want to delete.

You cannot delete an IAM role which still has policies attached to it. So after you have identified the relevant role, let's list the attached policies of that role.

In [19]:
iam.list_attached_role_policies(
    RoleName = role_name
)

{'AttachedPolicies': [{'PolicyName': 'AmazonS3FullAccess',
   'PolicyArn': 'arn:aws:iam::aws:policy/AmazonS3FullAccess'},
  {'PolicyName': 'AmazonPersonalizeFullAccess',
   'PolicyArn': 'arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess'}],
 'IsTruncated': False,
 'ResponseMetadata': {'RequestId': '4c15e0bf-bf81-4004-b7ca-d43952677219',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4c15e0bf-bf81-4004-b7ca-d43952677219',
   'content-type': 'text/xml',
   'content-length': '728',
   'date': 'Mon, 01 Feb 2021 19:16:32 GMT'},
  'RetryAttempts': 0}}

You need to detach the policies in the result above using the code below. Repeat for each attached policy.

In [20]:
iam.detach_role_policy(
    RoleName = role_name,
    PolicyArn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
)

{'ResponseMetadata': {'RequestId': 'cecb7aad-bd02-45f3-9518-682ae0bebd61',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'cecb7aad-bd02-45f3-9518-682ae0bebd61',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Mon, 01 Feb 2021 19:16:47 GMT'},
  'RetryAttempts': 0}}

In [21]:
iam.detach_role_policy(
    RoleName = role_name,
    PolicyArn = 'arn:aws:iam::aws:policy/AmazonS3FullAccess'
)

{'ResponseMetadata': {'RequestId': '81e0ae58-b145-4847-82d1-3ae83baa356c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '81e0ae58-b145-4847-82d1-3ae83baa356c',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Mon, 01 Feb 2021 19:16:50 GMT'},
  'RetryAttempts': 0}}

Finally, you should be able to delete the IAM role.

In [22]:
iam.delete_role(
    RoleName = role_name
)

{'ResponseMetadata': {'RequestId': '4504b5a7-7cb8-429f-b649-9bd15d6a7f87',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4504b5a7-7cb8-429f-b649-9bd15d6a7f87',
   'content-type': 'text/xml',
   'content-length': '200',
   'date': 'Mon, 01 Feb 2021 19:16:53 GMT'},
  'RetryAttempts': 0}}

To delete an S3 bucket, it first needs to be empty. The easiest way to delete an S3 bucket, is just to navigate to S3 in the AWS console, delete the objects in the bucket, and then delete the S3 bucket itself.